## Have Customers Narrow Their Travel Searches Based on Temperature and Precipitation

In [1]:
import pandas as pd
import os
import re
from config import g_key
import requests
import gmaps

Import the WeatherPy_vacation.csv file from Part 1 as a new DataFrame

In [2]:
# Output file
infolder = "weather_data"
infile = "WeatherPy_challenge.csv"

weather_data_df = pd.read_csv(os.path.join(infolder,infile))
weather_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,Pitimbu,BR,-7.47,-34.81,82.40,88,75,6.93,broken clouds,0.0,0.0
1,Rikitea,PF,-23.12,-134.97,73.85,74,100,25.17,light rain,0.0,0.0
2,Nambucca Heads,AU,-30.65,153.00,62.60,67,90,11.41,overcast clouds,0.0,0.0
3,Bara,NG,10.37,10.73,82.92,56,100,4.56,overcast clouds,0.0,0.0
4,Barrow,US,71.29,-156.79,30.20,92,90,12.75,mist,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
569,Leningradskiy,RU,69.38,178.42,29.39,97,99,6.02,overcast clouds,0.0,0.0
570,Ellisras,ZA,-23.66,27.74,59.81,37,0,0.92,clear sky,0.0,0.0
571,Lufilufi,WS,-13.87,-171.60,87.01,70,14,4.00,few clouds,0.0,0.0
572,Bac Lieu,VN,9.29,105.72,84.79,73,100,4.65,overcast clouds,0.0,0.0


Filter the DataFrame for minimum and maximum temperature preferences, and if the rain or snow accumulation is 0 inches or not using conditional statements.

In [3]:
# Constants
yesno_regex = "^(yes|no)$"

In [4]:
#Prompt the customer for the minimum temperature preference.

while True:
    try:
        min_str = input("\nWhat is the minimum temperature you would like for your trip? ")
        pref_temp_min = float(min_str)
        break
    except:
        print(f"\n<{min_str}> is an invalid temperature value. Please try again.")
        pass   


What is the minimum temperature you would like for your trip? 75


In [5]:
#Prompt the customer for the maximum temperature preference.

while True:
    try:
        max_str = input("\nWhat is the maximum temperature you would like for your trip? ")
        pref_temp_max = float(max_str)
        break
    except:
        print(f"\n<{max_str}> is an invalid temperature value. Please try again.")
        pass


What is the maximum temperature you would like for your trip? 85


In [6]:
# Prompt the customer to answer if he or she would like it to be raining or not

while True: 
    raining = input("Do you want it to be raining? (yes/no) ")
    raining = raining.lower()
    if re.search(yesno_regex, raining):
        raining = raining == "yes"
        break
    else:
        print("Please answer yes or no.")
    

Do you want it to be raining? (yes/no) no


In [7]:
# Prompt the customer to answer if he or she would like it to be snowing or not

# It can either be raining or snowing, but not both
if not raining:
    while True: 
        snowing = input("Do you want it to be snowing? (yes/no) ")
        snowing = snowing.lower()
        if re.search(yesno_regex, snowing):
            snowing = snowing == "yes"
            break
        else:
            print("Please answer yes or no.")
else:
    snowing = False


Do you want it to be snowing? (yes/no) no


In [8]:
# Filter dataframe 
city_data_df = weather_data_df.loc[(weather_data_df["Max Temp"] <= pref_temp_max) &
                              (weather_data_df["Max Temp"] >= pref_temp_min) ].copy().reset_index(drop=True)

if raining and not snowing:
    city_data_df = city_data_df[city_data_df["Rain (inches)"] > 0]
    city_data_df = city_data_df[city_data_df["Snow (inches)"] == 0]
elif snowing and not raining:
    city_data_df = city_data_df[city_data_df["Snow (inches)"] > 0]
    city_data_df = city_data_df[city_data_df["Rain (inches)"] == 0]
else:
    city_data_df = city_data_df[city_data_df["Rain (inches)"] == 0]
    city_data_df = city_data_df[city_data_df["Snow (inches)"] == 0]
    

In [9]:
city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,Pitimbu,BR,-7.47,-34.81,82.40,88,75,6.93,broken clouds,0.0,0.0
1,Bara,NG,10.37,10.73,82.92,56,100,4.56,overcast clouds,0.0,0.0
2,Pundaguitan,PH,6.37,126.17,83.57,78,100,9.08,overcast clouds,0.0,0.0
3,Pozo Colorado,PY,-23.49,-58.80,77.14,28,0,3.71,clear sky,0.0,0.0
4,Takoradze,GH,4.88,-1.76,81.28,80,100,7.83,overcast clouds,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
166,Kunduz,AF,36.73,68.86,82.60,18,0,3.02,clear sky,0.0,0.0
167,Arraial do Cabo,BR,-22.97,-42.03,78.80,69,40,4.70,scattered clouds,0.0,0.0
168,Bambanglipuro,ID,-7.95,110.28,78.12,87,100,3.09,light rain,0.0,0.0
169,Bac Lieu,VN,9.29,105.72,84.79,73,100,4.65,overcast clouds,0.0,0.0


Code is written to search
for a hotel using a Google
API and the JSON data is
retrieved.

In [10]:
def get_hotel(lat,lng):
    
    # Use base URL to search for hotels
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # params ... missing location
    params_temp = {"radius": 5000,
    "types": "lodging",
    "key": g_key}
    
    hotels = []
    
    for index in range(len(lat)):
        params = params_temp
        params["location"] = f"{lat[index]},{lng[index]}"
    
        # Make request and get the JSON data from the search.
        hotels.append(requests.get(base_url, params=params).json())
        
    return hotels

In [11]:
# Create a new DataFrame with the following columns: ● City ● Country ● Max Temp ● Current Description ● Lat ● Lng
vacation_df = city_data_df[["City","Country","Max Temp","Current Description","Lat","Lng"]].copy()

# Create a location dataframe of the Lat and Lng values in the hotels_df dataframe
# loc = vacation_df[["Lat","Lng"]]

# Pass the lists to the get_hotel function. Get a list of json payload of hotel names for each location
hotels = get_hotel(vacation_df["Lat"],vacation_df["Lng"])

# Loop through the hotels json list and retrieve the first hotel name for each location 
hotels = pd.Series(hotels)
hotel_names = []

# Iterate through the retrieves hotels json payload
for hotel in hotels:
    # A try/except block is used to add the hotel to the new DataFrame, and the IndexError is resolved.
    try:
        hotel_name = hotel["results"][0]["name"]
        hotel_names.append(hotel_name)
    except (IndexError):
        hotel_names.append("NA")

# Create the Hotel Name column in the hotels_df dataframe
vacation_df["Hotel Name"] = hotel_names

vacation_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Pitimbu,BR,82.40,broken clouds,-7.47,-34.81,Reserva do Abiaí
1,Bara,NG,82.92,overcast clouds,10.37,10.73,NA
2,Pundaguitan,PH,83.57,overcast clouds,6.37,126.17,Leonardo Family Beach House 1
3,Pozo Colorado,PY,77.14,clear sky,-23.49,-58.80,Hospedaje la Amistad
4,Takoradze,GH,81.28,overcast clouds,4.88,-1.76,Raybow International Hotel
...,...,...,...,...,...,...,...
166,Kunduz,AF,82.60,clear sky,36.73,68.86,قريشي مارکيٹ
167,Arraial do Cabo,BR,78.80,scattered clouds,-22.97,-42.03,Pousada Porto Praia
168,Bambanglipuro,ID,78.12,light rain,-7.95,110.28,Puri Brata
169,Bac Lieu,VN,84.79,overcast clouds,9.29,105.72,Khách sạn Sài Gòn Bạc Liêu


In [12]:
# Save and upload the new DataFrame as WeatherPy_vacation.csv

outfolder = "weather_data"
outfile = "WeatherPy_vacation.csv"

vacation_df.to_csv(os.path.join(outfolder,outfile),index=False)


Creates a marker layer
map with a pop-up marker
for each city that includes:
● Hotel name
● City
● Country
● Current weather
description with the
maximum
temperature

In [13]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = vacation_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# Create then show the marker layer on the Google map
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))